In [3]:
import os
import time
import pandas as pd
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
def get_acum(pm1, pm2, Temp_df):
    
    Temp_df['PM1'] = Temp_df['Average'].rolling(pm1).mean()
    Temp_df['PM2'] = Temp_df['Average'].rolling(pm2).mean()
    Temp_df.dropna(inplace = True)
    Temp_df['Posicion'] = np.where(Temp_df['PM1'] > Temp_df['PM2'], 1, -1)
    Temp_df['Retornos'] = np.log(Temp_df['Average']/Temp_df['Average'].shift(1))
    Temp_df.dropna(inplace = True)
    Temp_df['Estrategia'] = Temp_df['Retornos']*Temp_df['Posicion'].shift(1)
    Temp_df.dropna(inplace = True)
    # Temp_df['Retacum'] = Temp_df['Retornos'].cumsum().apply(np.exp)
    Temp_df['Estracum'] = Temp_df['Estrategia'].cumsum().apply(np.exp)
    
    return Temp_df['Estracum'].tail(1).copy()

In [5]:
def best_strategy(pm1, pm2, Bitso):
    
    Bitso['PM1'] = Bitso['Average'].rolling(pm1).mean()
    Bitso['PM2'] = Bitso['Average'].rolling(pm2).mean()
    Bitso.dropna(inplace = True)
    Bitso.plot(figsize=(12,8))
    Bitso['Posicion'] = np.where(Bitso['PM1'] > Bitso['PM2'], 1, -1)
    Bitso.plot(figsize=(12,8), secondary_y = 'Posicion')
    Bitso['Retornos'] = np.log(Bitso['Average']/Bitso['Average'].shift(1))
    Bitso.dropna(inplace = True)
    Bitso['Estrategia'] = Bitso['Retornos']* Bitso['Posicion'].shift(1)
    Bitso.dropna(inplace = True)
    Bitso['Retacum'] = Bitso['Retornos'].cumsum().apply(np.exp)
    Bitso['Estracum'] = Bitso['Estrategia'].cumsum().apply(np.exp)

    #------------------Print it out
    fig = plt.figure(figsize=(14,14))

    ax1 = plt.subplot2grid((25,1),(0,0),rowspan = 8, colspan = 1)
    Bitso['Average'].plot(title = ('Estrategia PMs ' + str(pm1*10) + ', ' + str(pm2*10)))
    Bitso['PM1'].plot(ax=ax1)
    Bitso['PM2'].plot(ax=ax1)

    ax1.set_ylabel('Ultimo Precio, PM1, PM2')
    plt.xticks([],[])

    ax2 = plt.subplot2grid((25,1),(8,0), rowspan = 3, colspan = 1)
    Bitso['Posicion'].plot(ylim=[-1.1,1.1])
    ax2.set_ylabel('Posicion')
    plt.xticks([],[])

    ax3=plt.subplot2grid((25,1),(11,0),rowspan = 8, colspan = 1)
    Bitso[['Retacum', 'Estracum']].plot(ax = ax3)
    ax3.set_ylabel('Retornos Acumulados')
    plt.xticks([],[])
    
    return Bitso

In [6]:
'''
Best So Far:
PM1: 1099
PM2: 1145
Score: 4.1813

7 DIAS:
PM1: 100, 127, 139, 142, 157, 250, 598, 631, 712, 733, 742, 763, 769, 772, 775, 778, 781, 787, 847, 958, 1096, 1135, 1180
PM2: 251, 211, 211, 211, 161, 291, 621, 1161, 1241, 1061, 1045, 1051, 911, 1061, 1061, 1061, 821, 881, 1161, 1101, 1201, 1181
Score: 1.16, 1.11, 1.12, 1.16, 1.19, 1.08, 1.08, 1.09, 1.09, 1.1, 1.1, 1.1, 1.1, 1.11, 1.11, 1.11, 1.15, 1.1, 1.08, 1.12, 1.13
'''

'\nBest So Far:\nPM1: 1099\nPM2: 1145\nScore: 4.1813\n\n7 DIAS:\nPM1: 100, 127, 139, 142, 157, 250, 598, 631, 712, 733, 742, 763, 769, 772, 775, 778, 781, 787, 847, 958, 1096, 1135, 1180\nPM2: 251, 211, 211, 211, 161, 291, 621, 1161, 1241, 1061, 1045, 1051, 911, 1061, 1061, 1061, 821, 881, 1161, 1101, 1201, 1181\nScore: 1.16, 1.11, 1.12, 1.16, 1.19, 1.08, 1.08, 1.09, 1.09, 1.1, 1.1, 1.1, 1.1, 1.11, 1.11, 1.11, 1.15, 1.1, 1.08, 1.12, 1.13\n'

In [30]:
# Gross Tunning

Best_Stra = 0
Best_PM1 = 0
Best_PM2 = 0
Start_PM1 = 40  # 380
Start_PM2 = 40  # 381
Last_PM1 = 150 # 1 dia = 1440, 30 dias = 43200 min
Last_PM2 = 180
Steps_PM1 = 1
Steps_PM2 = 1

the_path = os.getcwd()
data_path = os.path.join(the_path, '../data/processed-data/')
file_name = 'resampled_60min.csv'
file_path = data_path + file_name
df = pd.read_csv(file_path, index_col='Date')

df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.index = pd.to_datetime(df.index)
df_M = df.copy()
print(df_M.shape)

#fig = plt.figure()
#ax = fig.add_subplot(111)
#plt.ion()

#fig.show()
#fig.canvas.draw()
#i = 0
#L_i = []
#L_v = []

#For cycle para ver cual es la mejor combinacion

for pm1 in np.arange(Start_PM1, Last_PM1, Steps_PM1):
    for pm2 in np.arange(Start_PM2, Last_PM2, Steps_PM2):
        if (pm1<pm2) and (pm1/pm2>0.35):
            # i += 1
            Val_Stra = get_acum(pm1, pm2, df_M.copy())
            # L_i.append(i)
            # L_v.append(Val_Stra)
            # ax.clear()
            # ax.plot(L_i,L_v)
            # fig.canvas.draw()
            # print ('\rActual || PM1: {} | PM2: {} | Score: {} || Best || PM1: {} | PM2: {} | Score: {}'.format(pm1,pm2,Val_Stra[0],Best_PM1,Best_PM2,Best_Stra),end='')
            print ('\rActual || PM1: {} | PM2: {} | Score: {}'.format(pm1,pm2,Val_Stra[0]),end='')
            if (Val_Stra[0] > Best_Stra):
                Best_Stra = Val_Stra[0]
                Best_PM1 = pm1
                Best_PM2 = pm2
                print ('\n')
print('\nBEST: {}'.format(Best_Stra))

(4000, 1)
Actual || PM1: 40 | PM2: 41 | Score: 1.5046292781354245

Actual || PM1: 40 | PM2: 53 | Score: 1.5657430787875577

Actual || PM1: 40 | PM2: 56 | Score: 1.8528035659194668

Actual || PM1: 40 | PM2: 62 | Score: 2.1066086504401804

Actual || PM1: 41 | PM2: 64 | Score: 2.19717181655102775

Actual || PM1: 43 | PM2: 56 | Score: 2.33827573858125168

Actual || PM1: 43 | PM2: 64 | Score: 2.5457397551601887

Actual || PM1: 44 | PM2: 56 | Score: 2.93745106463570065

Actual || PM1: 115 | PM2: 166 | Score: 2.95577998858622956

Actual || PM1: 116 | PM2: 167 | Score: 2.9940180317405587

Actual || PM1: 122 | PM2: 152 | Score: 3.0414799769297842

Actual || PM1: 122 | PM2: 153 | Score: 3.1254840235748187

Actual || PM1: 122 | PM2: 155 | Score: 3.227128725188943

Actual || PM1: 123 | PM2: 154 | Score: 3.2780567697422814

Actual || PM1: 124 | PM2: 154 | Score: 3.3103976110583586

Actual || PM1: 130 | PM2: 135 | Score: 3.3627907723941813

Actual || PM1: 130 | PM2: 136 | Score: 3.698933040624883

A

 5 min = PM1: 1103 | PM2: 1107 | Score: 1.3046169725292444
10 min = PM1: 463 | PM2: 486 | Score: 1.617775237893657
15 min = PM1: 209 | PM2: 215 | Score: 1.8822113944465655
30 min = PM1: 103 | PM2: 110 | Score: 2.1155620079972404
60 min = PM1: 130 | PM2: 136 | Score: 3.698933040624883
         PM1: 44  | PM2: 56  | Score: 2.93745106463570065


    Freq: 1 min -> 2.78 days
    Freq: 5 min -> 13.89 days
    Freq: 10 min -> 27.78 days
    Freq: 15 min -> 41.67 days
    Freq: 30 min -> 83.33 days
    Freq: 60 min -> 166.67 days
    Freq: 2 hrs -> 333.33 days
    Freq: 3 hrs -> 1.36 years
    Freq: 5 hrs -> 2 years
    Freq: 8 hrs -> 2 years
    Freq: 12 hrs -> 2 years
    Freq: 24 hrs -> 2 years
